<a href="https://colab.research.google.com/github/Jesteban247/AI-Driven-Gym-and-Nutrition-Workflow-with-CrewAI-and-IBM-Watson/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Explanation of Workflow**

1. **Gym Researcher**: Searches for scientific studies on push/pull/leg hypertrophy training for 6 days.
2. **Gym Trainer**: Organizes that research into a structured training plan.
3. **Nutrition Researcher**: Finds scientifically backed bulking and cutting diets.
4. **Dietician**: Develops a detailed 7-day meal plan based on the research.

This system will automate the process of fetching information and creating your personalized training and nutrition plans. When the code runs, it uses the Serper search tool to pull information from the web, and the LLM agents process the data to create the training and nutrition plans.

# **Section 1: Install Required Packages**

In this section, we install the necessary Python packages:

- **`crewai`**: A framework to create AI-driven workflows where agents can perform tasks.
- **`crewai_tools`**: Includes tools such as internet search (Serper), which agents use to fetch information.
- **`langchain_ibm`**: A library that allows interaction with IBM Watson’s language models.

In [ ]:
!pip install crewai crewai_tools langchain_ibm

# **Section 2: Import Libraries**

Here, we import the libraries required to set up and run the agents:

- **`Crew, Task, Agent`**: These are part of the `crewai` package, where:
  - `Crew` orchestrates agents and tasks.
  - `Task` defines the work that needs to be done.
  - `Agent` is the entity performing the task.
- **`SerperDevTool`**: A tool for searching the web using the Serper API.
- **`WatsonxLLM`**: A class from `langchain_ibm` that allows interaction with IBM Watson's large language models (LLMs).


In [1]:
from crewai import Crew, Task, Agent
from crewai_tools import SerperDevTool
from langchain_ibm import WatsonxLLM
import os

In [23]:
# Authenticate Google Colab and get stored credentials for APIs
from google.colab import userdata
serper=userdata.get('serper')
watson=userdata.get('watson')
project_id=userdata.get('project_id')

# Setting up API keys for Watson and Serper
os.environ["WATSONX_APIKEY"] = watson
os.environ["SERPER_API_KEY"] = serper
os.environ["PROJECT_ID"] = project_id

# **Section 3: Define Parameters and Create Language Models**

In this section, we define the language models (LLMs) the agents will use.

- **Parameters**: These control how the LLM generates text.
  - **`decoding_method`**: Controls how the LLM selects the next word (we use "greedy," which selects the highest probability word at each step).
  - **`max_new_tokens`**: Limits the number of new tokens (words) the model can generate.

The models are:
- **`meta-llama/llama-3-70b-instruct`**: A large language model that specializes in instruction-based tasks.
- **`mistralai/mistral-large`**: Another LLM focused on function calling and structured tasks.

In [25]:
parameters = {"decoding_method": "greedy", "max_new_tokens": 500}

# General purpose language model
llm = WatsonxLLM(
    model_id="meta-llama/llama-3-70b-instruct",
    url="https://us-south.ml.cloud.ibm.com",
    params=parameters,
    project_id=project_id
)

# Model for function calling tasks
function_calling_llm = WatsonxLLM(
    model_id="mistralai/mistral-large",
    url="https://us-south.ml.cloud.ibm.com",
    params=parameters,
    project_id=project_id
)

# **Section 4: Set Up Tools and Create Agents**

Here we define the agents:
1. **Researcher (Gym Split)**: This agent will search for a 6-day gym push/pull/leg split and retrieve exercise advice from studies on hypertrophy (muscle growth).
2. **Trainer (Gym Planner)**: This agent, an expert in bodybuilding, will use the research data to plan the sets, reps, exercises, and rest time for maximum hypertrophy.
3. **Nutrition Researcher**: This agent will search for the best diet plans for bulking and cutting based on scientific research.
4. **Dietician (Meal Planner)**: This agent will design a 7-day meal plan based on the best practices found for bulking and cutting.

In [26]:
# Web search tool using Serper
search = SerperDevTool()

In [27]:
# Agent 1: Researcher (Gym Split)
gym_researcher = Agent(
    llm=llm,
    function_calling_llm=function_calling_llm,
    role="Gym Researcher",
    goal="Find the best 6-day gym push/pull/leg split for hypertrophy based on scientific research.",
    backstory="You are a fitness expert with knowledge in hypertrophy and bodybuilding.",
    allow_delegation=False,
    tools=[search],
    verbose=1,
)

# Agent 2: Trainer (Gym Planner)
gym_trainer = Agent(
    llm=llm,
    role="Bodybuilding Trainer",
    goal="Create a detailed workout plan for 6 days focusing on hypertrophy, with exercises, sets, reps, and rest times.",
    backstory="You are an experienced bodybuilding coach who knows how to optimize training for muscle growth.",
    allow_delegation=False,
    verbose=1,
)

# Agent 3: Nutrition Researcher
nutrition_researcher = Agent(
    llm=llm,
    function_calling_llm=function_calling_llm,
    role="Nutrition Researcher",
    goal="Find the best foods and strategies for bulking and cutting based on scientific research.",
    backstory="You are a sports nutritionist with expertise in diet optimization for bodybuilders.",
    allow_delegation=False,
    tools=[search],
    verbose=1,
)

# Agent 4: Dietician (Meal Planner)
dietician = Agent(
    llm=llm,
    role="Nutritionist",
    goal="Create a detailed 7-day meal plan for both bulking and cutting, with food portions, timings, and macronutrients.",
    backstory="You are an expert in nutrition with a focus on bodybuilding.",
    allow_delegation=False,
    verbose=1,
)


# **Section 5: Define and Run Tasks**

We assign tasks to each agent.

- **Gym Researcher Task**: Find information about push/pull/leg hypertrophy training based on studies.
- **Gym Trainer Task**: Organize the workout split based on the researcher's findings.
- **Nutrition Research Task**: Gather information about bulking and cutting diets.
- **Dietician Task**: Create meal plans based on nutrition research.

In [28]:
# Task 1: Gym Split Research
task1 = Task(
    description="Find scientific studies and research for a 6-day push/pull/leg gym split for hypertrophy.",
    expected_output="A bullet point summary of exercises, sets, reps, and rest times for each day.",
    output_file="gym_research.txt",
    agent=gym_researcher,
)

# Task 2: Gym Workout Plan
task2 = Task(
    description="Create a 6-day gym workout plan for hypertrophy based on the researcher's findings.",
    expected_output=(
        "A concise training plan with the top exercises for each day of the split. "
        "Use the format: Exercise - Sets x Reps, Rest Time."
    ),
    output_file="gym_plan.txt",
    agent=gym_trainer,
)

# Task 3: Nutrition Research for Bulking and Cutting
task3 = Task(
    description="Research the best diets and foods for bulking and cutting based on scientific evidence.",
    expected_output="A summary of ideal foods and strategies for bulking and cutting.",
    output_file="nutrition_research.txt",
    agent=nutrition_researcher,
)


# Task 4: 7-Day Meal Plan
task4 = Task(
    description="Create a 7-day meal plan for both bulking and cutting based on the nutrition research.",
    expected_output=(
        "A concise meal plan for bulking and cutting, listing the best foods for each meal. "
        "Use the format: Meal 1 (Breakfast): [Food Item], [Serving Size]."
    ),
    output_file="meal_plan.txt",
    agent=dietician,
)

In [29]:
# Combine agents and tasks into a crew
crew = Crew(agents=[gym_researcher, gym_trainer, nutrition_researcher, dietician], tasks=[task1, task2, task3, task4], verbose=1)
print(crew.kickoff())

 [2024-09-13 05:39:41][DEBUG]: == Working Agent: Gym Researcher
 [2024-09-13 05:39:41][INFO]: == Starting Task: Find scientific studies and research for a 6-day push/pull/leg gym split for hypertrophy.


> Entering new CrewAgentExecutor chain...
I need to find scientific studies and research for a 6-day push/pull/leg gym split for hypertrophy.

Action: Search the internet
Action Input: {"search_query": "scientific studies on 6-day push/pull/leg gym split for hypertrophy"}
Observation 


Search results: Title: 6 Day Per Week Push/Pull/Legs Hypertrophy Split - Weightology
Link: https://weightology.net/muscle-gain/6-day-per-week-push-pull-legs-hypertrophy-split/
Snippet: The 6-day per week push/pull/legs split is similar to the 4-day per week upper/lower split, but you divide the upper body workout into a push muscle day.
---
Title: The Push/Pull/Legs Routine for Muscle Gains | Aston University
Link: https://www.aston.ac.uk/sport/news/tips/fitness-exercise/push-pull-legs
Snippet: The push

# **Observations**

1. **Current Parameters**:
   - **Decoding Method**: Using `greedy` decoding ensures straightforward and predictable responses, but it may lack creativity and diversity. For more varied outputs, exploring methods like beam search or sampling could be beneficial.
   - **Max New Tokens**: The current limit of 500 tokens per response is practical for managing costs and ensuring quick responses. However, this can lead to incomplete answers, especially for detailed tasks.

2. **Usage and Token Management**:
   - Given the extensive research and writing tasks handled by four agents, the total token usage has reached 20,556 tokens. This is substantial but manageable within the constraints of a free trial.
   - With a free trial limit of 50,000 tokens, it is crucial to balance the length of responses with the total number of tokens used.

3. **Improving Results**:
   - **Increasing Token Limit**: If additional tokens are available or if you upgrade your plan, increasing the `max_new_tokens` limit could enhance response completeness. Testing with a limit of 2,000 tokens might provide more detailed answers.
   - **Prompt Optimization**: Refining prompts to be more specific can help manage token usage efficiently and improve the quality of responses.
   - **Exploring Decoding Methods**: Experimenting with alternative decoding methods could yield more diverse and insightful outputs.

#### **Conclusion**

The current setup with a token limit of 500 is effective for controlling costs and generating concise responses. However, for more comprehensive results, especially in complex research and writing tasks, increasing the token limit would be advantageous. For now, managing token usage carefully while optimizing prompts and exploring decoding methods can help maximize the efficiency and effectiveness of the AI agents.